In [6]:
import re
import numpy as np
import pandas as pd

# Read the CSV file
df = pd.read_csv('Dataframe_Boligsiden_Distance.csv', encoding='utf-8', sep=';')
import numpy as np

# Columns to clean
columns_to_clean = ['price', 'udbetaling', 'area', 'land area', 'rooms', 'ejerudg', 'constructed', 'days active']

# Function to clean the columns
def clean_column_values(column_value):
    # Remove all characters except numbers
    cleaned_value = ''.join(filter(str.isdigit, str(column_value)))
    # Remove any occurrence of '²' character (if present)
    cleaned_value = cleaned_value.replace('²', '')
    # If value is empty or '-' after cleaning, set it to NaN
    if cleaned_value == '' or cleaned_value == '-':
        return np.nan
    return int(cleaned_value)

# Apply the cleaning function to the specified columns
for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(clean_column_values)


def convert_nan_to_bool(value):
    return 0 if np.isnan(value) else 1

# Display a sample of the cleaned data
def clean_energy_column_v3(value):
    # Remove the "Energimærke" text
    cleaned_value = str(value).replace('Energimærke', '').strip()
    # Extract the letter between A and G using regex
    match = re.search(r'[A-G]', cleaned_value)
    return match.group(0) if match else np.nan

# Apply the updated cleaning function to the 'energy' column
df['energy'] = df['energy'].apply(clean_energy_column_v3)
df['city_3'] = df['city']
df['city'] = df['city_2']
# Split the column at ':'
df['seller'] = df['days at current seller'].str.split(':', expand=True)[0]

# Clean the right part and assign it back
df['days at current seller'] = df['days at current seller'].str.split(':', expand=True)[1].str.extract('(\d+)')[0]

# Convert 'days at current seller' column to numeric type
df['days at current seller'] = pd.to_numeric(df['days at current seller'], errors='coerce')
# Adjusting the extraction process to handle comma as decimal separator
# Extract residents
df['residents'] = df['extra'].str.extract(r'Der er ([\d\.]+) indbyggere')

# Convert the 'residents' column to the appropriate data type after removing dots
df['residents'] = df['residents'].str.replace('.', '').astype(float)

# Extract municipal
df['municipal'] = df['extra'].str.extract(r'indbyggere i (.*?) Kommune')

# Convert comma to dot for 'municipal tax rate' and 'grundskyldspromille' columns
df['municipal tax rate'] = df['extra'].str.extract(r'Kommuneskatten er på ([\d,]+)%').replace(',', '.', regex=True)
df['municipal tax rate'] = df['municipal tax rate'].astype(float)

# Adjusting the regex pattern to capture the part between 'grundskyldspromillen på ' and '‰'
df['grundskyldspromille'] = df['extra'].str.extract(r'grundskyldspromillen på (.*?)‰')
df['grundskyldspromille'] = df['grundskyldspromille'].str.replace(',', '.')
# Display some of the non-NaN values for 'grundskyldspromille'
non_nan_values_updated = df[df['grundskyldspromille'].notna()]['grundskyldspromille']

# Simplify the categories of homes
target_values = ['andel','villalejlighed','lejlighed','villa', 'ejerlejlighed','rækkehus','fritidsbolig','landejendom']
# Create a regex pattern from target_values
pattern = '(' + '|'.join(map(re.escape, target_values)) + ')'
# Extract matching values using str.extract and create 'energi' column
df['property_type'] = df['building type'].str.lower().str.extract(pattern, expand=False)

# Define a dictionary to map values to target values
porperty_type_mapping = {
    'villa': 'Villa',
    'lejlighed':'Ejerlejlighed',
    'andel':'Andel',
    'ejerlejlighed':'Ejerlejlighed',
    'rækkehus':'Villa',
    'fritidsbolig':'Villa',
    'landejendom':'Villa',
    'villalejlighed':'Ejerlejlighed'
}

# Map the values in the 'property_type' column to target values
df['property_type'] = df['property_type'].map(porperty_type_mapping)

# Drop rows with NaN values in the 'Property_type' variable:
df = df.dropna(subset=['price','rooms','ejerudg','energy'])

# Remove observations without a geo code
df = df[df['latitude'] != 'Not available']

# States that 'land area' should be 0 if the value is NaN
df['land area'] = df['land area'].apply(lambda x: 0 if np.isnan(x) else x)

df = df[df['area'] > 10]
df = df[df['rooms'] < 40]
df = df[df['distance to city'] < 100]


# Calculates price per m2. 
df['sqm_price'] = df['price'] / df['area']
# Calculate age of a house
df['Age'] = 2023 - df['constructed']

# Drop columns with insufficient data availability
columns_to_drop = df.columns[df.count() < 20000]
df = df.drop(columns=columns_to_drop)

df = df.drop(columns=['road','building type','udbetaling','seller', 'full_address', 'zip_address',
                            'link to seller', 'city_3', 'zip address', 'city_2', 'zip',
                           'extra', 'constructed', 'days at current seller'])


for column in df.columns:
    print(f"Number of non-null values in '{column}': {df[column].count()}")



C:\Users\Marti\AppData\Local\Temp\ipykernel_9732\2752161858.py:57: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['residents'] = df['residents'].str.replace('.', '').astype(float)


Number of non-null values in 'price': 26527
Number of non-null values in 'city': 26527
Number of non-null values in 'energy': 26527
Number of non-null values in 'area': 26527
Number of non-null values in 'land area': 26527
Number of non-null values in 'rooms': 26527
Number of non-null values in 'ejerudg': 26527
Number of non-null values in 'days active': 26527
Number of non-null values in 'latitude': 26527
Number of non-null values in 'longitude': 26527
Number of non-null values in 'distance to Copenhagen': 26527
Number of non-null values in 'distance to Aarhus': 26527
Number of non-null values in 'distance to Aalborg': 26527
Number of non-null values in 'distance to Odense': 26527
Number of non-null values in 'distance to city': 26527
Number of non-null values in 'closest city': 26527
Number of non-null values in 'residents': 26527
Number of non-null values in 'municipal': 26527
Number of non-null values in 'municipal tax rate': 26527
Number of non-null values in 'grundskyldspromille'

In [7]:
df.to_csv('Final_Cleaned.csv', encoding='utf-8', sep=';', index=False)